In [41]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("bart_predictions.csv")

In [3]:
df

,Unnamed: 0,Generated Text,Actual Text
0,0,summarize: James Veitch is a hedge fund execut...,summarize: Suspicious emails: unclaimed insura...
1,1,"summarize: ""I collect stories, and I collect d...",summarize: BrenÃ© Brown studies human connecti...
2,2,summarize: Can you draw like Michelangelo? In ...,"summarize: Most people think they can't draw, ..."
3,3,summarize: What is it about my brother's brain...,summarize: Jill Bolte Taylor got a research op...
4,4,"summarize: In the 1970s, a woman was diagnosed...",summarize: Career analyst Dan Pink examines th...
...,...,...,...
811,811,"summarize: In many places in the world, we don...",summarize: Ever have the sneaking suspicion yo...
812,812,"summarize: What if we could create amazing, in...",summarize: When architect AntÃ³n GarcÃ­a-Abril...
813,813,"summarize: In this inspiring talk, Sarah Parca...",summarize: Sarah Parcak hoped the power of the...
814,814,"summarize: In this short, inspiring talk, Nati...",summarize: Estamos pescando mÃ¡s rÃ¡pido que l...


In [33]:
from rouge_score import rouge_scorer 
# take 3 n gram models and rougeL and rougeLsum types of score
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rouge3','rougeL','rougeLsum'], use_stemmer=True)

def get_rougue_score(text, highlights, metric="rougeL"):
  max_score = 0
  for h_text in highlights:
    score =  scorer.score(text, h_text)[metric].fmeasure
    if score > max_score:
      max_score = score 
  return max_score 

In [12]:
df['Rouge1'] = df.apply(lambda x: get_rougue_score(x['Actual Text'], x['Generated Text'], metric="rouge1"), axis=1)
df['Rouge2'] = df.apply(lambda x: get_rougue_score(x['Actual Text'], x['Generated Text'], metric="rouge2"), axis=1)
df['Rouge3'] = df.apply(lambda x: get_rougue_score(x['Actual Text'], x['Generated Text'], metric="rouge3"), axis=1)
df['RougeL'] = df.apply(lambda x: get_rougue_score(x['Actual Text'], x['Generated Text'], metric="rougeL"), axis=1)
df['RougeLsum'] = df.apply(lambda x: get_rougue_score(x['Actual Text'], x['Generated Text'], metric="rougeLsum"), axis=1)

In [37]:
df.to_csv("bart_predictions_with_rouge.csv", sep=',', encoding='utf-8', index=False)

# Run below for generating the bert score of the abstract model per column
## check the cummulative mean bert_score at the end

In [6]:
from datasets import list_metrics, load_metric
metrics_list = list_metrics()

In [4]:
df_abstract = pd.read_csv("yale_bart_predictions.csv")

In [43]:
bert_metric = load_metric("bertscore")
def bert_scorer(text, highlights, metric):
    minLen = min(len(text),len(highlights))
    text = text[:minLen]
    highlights = highlights[:minLen]
    return mean(metric.compute(predictions=highlights, references=text, lang="en", verbose=False)['f1'])


In [ ]:
df_abstract["bertscore"] = df_abstract.apply(lambda x: bert_scorer(x['Actual Text'],x['Generated Text'], bert_metric), axis=1)

In [ ]:
cummulative_bertscore = df_abstract["bertscore"].mean()
print("Cummulative Bert Score:",cummulative_bertscore)

In [ ]:
df_abstract.to_csv("yale_bart_predictions_bertscore.csv", sep=',', encoding='utf-8', index=False)